Code for Ucell Scoring taken from https://github.com/BoevaLab/ANS_signature_scoring/

In [6]:
from typing import Any, List, Optional, Tuple

def check_signature_genes(var_names: List[str], gene_list: List[str], return_type: Any = list) -> List[str]:
    """
    The method checks the availability of signature genes in the list of available genes (var_names). Genes not present in var_names are excluded.

    Args:
        var_names: List of available genes in the dataset.
        gene_list: List of genes to score for (signature).
        return_type: Indicates whether to return a list or a set.

    Returns:
        Filtered gene list.

    Raises:
        ValueError
            if return_type is not of type list or set.
            if gene_list gets empty.
    """
    # assume that if single string is passed it represents a single gene that should be scored
    if isinstance(gene_list, str):
        gene_list = [gene_list]

    if type(gene_list) not in [list, set]:
        raise ValueError(f"gene_list needs to be either list or set.")

    if len(gene_list) == 0:
        raise ValueError(f'gene_list must contain at least 1 gene name in adata.var_names')

    if len(gene_list) != len(set(gene_list)):
        seen = set()
        duplicates = [x for x in gene_list if x in seen or seen.add(x)]
        warnings.warn(f'The passed gene_list contains duplicated genes: {duplicates}')

    if return_type not in [list, set]:
        raise ValueError(f"return_type needs to be either list or set.")

    if not isinstance(var_names, list):
        var_names = var_names.tolist()

    genes_to_ignore = (set(gene_list)).difference(set(var_names))
    gene_list = (set(var_names)).intersection(set(gene_list))

    if len(genes_to_ignore) > 0:
        sc.logging.warning(f"genes are not in var_names and ignored: {sorted(list(genes_to_ignore))}")
    if len(gene_list) == 0:
        raise ValueError("No valid genes were passed for scoring.")

    return return_type(gene_list)

In [28]:
import warnings
from typing import Optional, List

import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from joblib import Parallel, delayed
from scanpy._utils import _check_use_raw
from scipy.sparse import issparse, csr_matrix, isspmatrix_csr
from scipy.stats import rankdata

def u_stat(rank_value, maxRank=1500):
    """
    The method computes the U statistic on signature gene ranks.

    Args:
        rank_value: Ranks of the signature genes.
        maxRank: Cutoff for maximum rank allowed.

    Returns:
        The U statistic for given signature gene ranks.
    """
    insig = rank_value > maxRank
    if all(insig):
        return 0
    else:
        rank_value[insig] = maxRank + 1
        rank_sum = rank_value.sum()
        len_sig = len(rank_value)
        u_value = rank_sum - (len_sig * (len_sig + 1)) / 2
        auc = 1 - u_value / (len_sig * maxRank)
        return auc


def compute_ranks_and_ustat(X_data, index, columns, gene_list, X_indices=None, X_indptr=None, X_shape=None,
                            maxRank=1500):
    """
    The following method computes for each cell in `X_data` the UCell score.

    Args:
        X_data: Current batch of gene expression data.
        index: Index of cells.
        columns: Names of genes.
        gene_list: Signature genes.
        X_indices: For sparse matrix reconstruction indices. If None, method assumes `X_data` to be a dense matrix.
        X_indptr: For sparse matrix reconstruction index pointers. If None, method assumes `X_data` to be a dense matrix.
        X_shape: For sparse matrix reconstruction shape of original matrix. If None, method assumes `X_data` to be
            a dense matrix.
        maxRank:  Cutoff for maximum rank allowed.

    Returns:
        For each cell in X_data the method returns the UCell score.
    """
    if any([x is None for x in [X_indices, X_indptr, X_shape]]):
        data_df = pd.DataFrame(
            X_data, index=index, columns=columns
        )
    else:
        data_df = pd.DataFrame(
            csr_matrix((X_data, X_indices, X_indptr), X_shape, copy=False).todense(), index=index, columns=columns
        )

    res = (data_df.apply(
        lambda x: rankdata(-x),
        axis=1,
        raw=True
    ))[gene_list]

    del data_df

    score = res.apply(
        func=(lambda x: u_stat(x, maxRank=maxRank)),
        axis=1,
        raw=False
    )
    return score


def score_genes(
        adata: AnnData,
        gene_list: List[str],
        maxRank: int = 1500,
        bs: int = 500,
        score_name: str = "UCell_score",
        random_state: Optional[int] = None,
        copy: bool = True,
        use_raw: Optional[bool] = None,
        verbose: int = 0,
        joblib_kwargs: dict = {'n_jobs': 4}
) -> Optional[AnnData]:
    """

    UCell signature scoring method is a Python implementation of the scoring method proposed by Andreatta et al. 2021.

    Massimo Andreatta and Santiago J Carmona. „UCell: Robust and scalable single-cell
    gene signature scoring“. en. In: Comput. Struct. Biotechnol. J. 19 (June 2021),
    pp. 3796–3798 (cit. on pp. iii, 2, 9, 15, 16).

    Implementation is inspired by score_genes method of Scanpy
    (https://scanpy.readthedocs.io/en/latest/generated/scanpy.tl.score_genes.html#scanpy.tl.score_genes)

    Args:
        adata: AnnData object containing the gene expression data.
        gene_list: A list of genes (signature) for which the cells are scored for.
        maxRank: Cutoff for maximum rank allowed.
        bs: The number of cells in a processing batch.
        score_name: Column name for scores added in `.obs` of data.
        random_state: Seed for random state.
        copy: Indicates whether original or a copy of `adata` is modified.
        use_raw: Whether to compute gene signature score on raw data stored in `.raw` attribute of `adata`
        verbose: If verbose is larger than 0, print statements are shown.
        joblib_kwargs: Keyword argument for parallel execution with joblib.

    Returns:
        If copy=True, the method returns a copy of the original data with stored UCell scores in `.obs`, otherwise
        None is returned.
    """
    start = sc.logging.info(f"computing score {score_name!r}")
    if verbose > 0:
        print(f"computing score {score_name!r}")

    adata = adata.copy() if copy else adata

    use_raw = _check_use_raw(adata, use_raw)

    _adata = adata.raw if use_raw else adata

    if random_state is not None:
        np.random.seed(random_state)

    # remove genes from gene_list not available in the data
    gene_list = check_signature_genes(_adata.var_names, gene_list)

    # check type of rank
    if not isinstance(maxRank, int):
        raise ValueError(f"maxRank {maxRank} must be of type int")

    # check maxRank is not larger than available nr. of genes
    if maxRank > len(_adata.var_names):
        print(
            f"Provided maxRank is larger than the number of available genes. Set maxRank=len(adata.var_names)"
        )
        maxRank = len(_adata.var_names)

    # check that signature is not longer than maxRank
    if maxRank < len(gene_list) <= len(_adata.var_names):
        warnings.warn(
            f"The provided signature contains more genes than the maxRank parameter. maxRank is increased to "
            f"signature length "
        )
        maxRank = len(gene_list)

    sparse_X = issparse(_adata.X)
    if sparse_X and not isspmatrix_csr(adata.X):
        adata.X = adata.X.tocsr()
        warnings.warn("Chaning adata.X format to CSR format")
        # create groups of managable sizes
    bss = pd.cut(np.arange(_adata.obs.shape[0]), (_adata.obs.shape[0] // bs + 1), labels=False)

    scores = Parallel(**joblib_kwargs)(
        delayed(compute_ranks_and_ustat)(
            X_data=_adata[group[1].index,].X.data if sparse_X else _adata[group[1].index,].X,
            X_indices=_adata[group[1].index,].X.indices if sparse_X else None,
            X_indptr=_adata[group[1].index,].X.indptr if sparse_X else None,
            X_shape=_adata[group[1].index,].X.shape if sparse_X else None,
            index=group[1].index,
            columns=_adata.var_names,
            gene_list=gene_list,
            maxRank=maxRank) for group in _adata.obs.groupby(bss))
    scores = pd.concat(scores)

    adata.obs[score_name] = scores

    sc.logging.info(
        "    finished",
        time=start,
        deep=("added\n" f"    {score_name!r}, score of gene set (adata.obs)."),
    )
    return adata if copy else None

In [37]:
adata = sc.read_h5ad('A01_RAW.h5ad')
adata

AnnData object with n_obs × n_vars = 10856 × 32146
    obs: 'batch', 'tissue', 'doublet', 'doublet_score'
    var: 'gene_ids', 'n_cells'

In [34]:
adata.var_names

Index(['OR4F5', 'RP11-34P13.7', 'RP11-34P13.8', 'RP11-34P13.9', 'FO538757.3',
       'FO538757.2', 'AP006222.2', 'RP5-857K21.15', 'RP4-669L17.2',
       'RP4-669L17.10',
       ...
       'AC011043.1', 'AL592183.1', 'AC007325.1', 'AC007325.4', 'AC007325.2',
       'BX072566.1', 'AL354822.1', 'AC023491.2', 'AC004556.1', 'AC240274.1'],
      dtype='object', length=32146)

In [38]:
gene_list = pd.read_csv('hallmark_surv_curated.csv')
gene_list

,IA,TPI,GIM,SPS,AID,ERI,RCD,EGS,DCE,AIM
0,CCR3,PLCB2,BLM,PAK2,HCST,E2F1,PAK2,PAK2,PRKAG3,CDH15
1,HIF1A,PRKCG,FANCI,GNG3,HIF1A,PIK3CD,IL6R,GNG3,NFE2L2,PAK2
2,PRKCG,TNFAIP3,RECQL4,IL6R,PLCB2,NRAS,PLK1,RBL1,GPI,PLCB2
3,MMP1,CARD11,POLD3,PLCB2,CXCR1,PIK3CG,ELK1,FZD2,HIF1A,FZD2
4,PIK3CD,GNB1,PCNA,PKN3,CCL26,PRKCA,BCL6,WNT3A,PFKP,WNT3A
...,...,...,...,...,...,...,...,...,...,...
202,NaN,NaN,NaN,PIM1,NaN,NaN,NaN,NaN,NaN,NaN
203,NaN,NaN,NaN,CSF3,NaN,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,NaN,BARD1,NaN,NaN,NaN,NaN,NaN,NaN
205,NaN,NaN,NaN,GNA12,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
for i in gene_list.columns:
    genes = gene_list[i].dropna().to_list()
    scored_adata = score_genes(adata=adata, gene_list=genes, score_name=i)
    adata=scored_adata

adata.obs

,batch,tissue,doublet,doublet_score,IA,TPI,GIM,SPS,AID,ERI,RCD,EGS,DCE,AIM
AAACCTGAGAAGGGTA-1-8,8,Adipose,0.0,3.101472e-64,0.052988,0.017619,0.052130,0.114141,0.045288,0.037139,0.103837,0.081081,0.167637,0.117675
AAACCTGAGACGCTTT-1-8,8,Adipose,0.0,1.113508e+00,0.073131,0.030940,0.040435,0.109809,0.061171,0.013375,0.087562,0.069977,0.149741,0.134420
AAACCTGAGGAGCGTT-1-8,8,Adipose,0.0,1.420106e-124,0.067012,0.036655,0.043667,0.102290,0.044514,0.036958,0.099073,0.080125,0.199126,0.125010
AAACCTGAGGGTTCCC-1-8,8,Adipose,0.0,1.420106e-124,0.069810,0.039405,0.040295,0.111040,0.062649,0.021833,0.096528,0.082146,0.159844,0.117536
AAACCTGAGTCGCCGT-1-8,8,Adipose,0.0,1.989470e+00,0.025381,0.028524,0.038965,0.097859,0.038207,0.012125,0.087302,0.067755,0.170074,0.113355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCACAAACC-1-8,8,Adipose,0.0,3.805684e-105,0.047667,0.000000,0.037032,0.109142,0.026631,0.000000,0.091725,0.088042,0.149948,0.107325
TTTGTCATCATGTCTT-1-8,8,Adipose,0.0,2.676102e+01,0.089512,0.052976,0.036958,0.106772,0.058486,0.048306,0.098496,0.078404,0.182881,0.130691
TTTGTCATCGTTTATC-1-8,8,Adipose,0.0,9.242906e-165,0.039976,0.060786,0.044863,0.097916,0.053757,0.000000,0.080582,0.058956,0.167267,0.122906
TTTGTCATCTACCAGA-1-8,8,Adipose,0.0,9.242906e-165,0.041952,0.067833,0.039660,0.110191,0.056171,0.000000,0.096659,0.070630,0.153407,0.126506
